In [1]:
import numpy as np
import pandas as pd
from datetime import date
from functools import reduce
import warnings

warnings.filterwarnings('ignore')

## Datasets Used

In the `data` folder, we include a `datasets_used.csv` to keep track of the datasets we find useful. Many datasets are updated regulary, so we also include the date we retrieved the dataset for better reproducibility. If the dataset is provided on Github, we include the link to the specific version we are using.

| name | local_path | url | date_retrieved | 
| :-- | :--- | :--- | :--- | 
| `covid-tracking-states-current` | covid-tracking/states_current.csv | https://github.com/COVID19Tracking/covid-tracking-data/blob/05363f00ed663709c54b066722094f966dddd1bc/data/states_current.csv  | 2020-04-23 |
| `covid-tracking-states-daily` | covid-tracking/states_daily_4pm_et.csv | https://github.com/COVID19Tracking/covid-tracking-data/blob/05363f00ed663709c54b066722094f966dddd1bc/data/states_daily_4pm_et.csv | 2020-04-23 | 
| `covid-tracking-us-current` | covid-tracking/us_current.csv | https://github.com/COVID19Tracking/covid-tracking-data/blob/05363f00ed663709c54b066722094f966dddd1bc/data/us_current.csv | 2020-04-23 | 
| `covid-tracking-us-daily` | covid-tracking/us_daily.csv | https://github.com/COVID19Tracking/covid-tracking-data/blob/05363f00ed663709c54b066722094f966dddd1bc/data/us_daily.csv | 2020-04-23 | 
| `khn-icu-beds-by-county` | khn/data-FPBfZ.csv  | https://khn.org/news/as-coronavirus-spreads-widely-millions-of-older-americans-live-in-counties-with-no-icu-beds/ | 2020-04-21 | 
| `khn-hospital-by-county` | khn/KHN_ICU_bed_county_analysis_2.csv | https://khn.org/wp-content/uploads/sites/2/2020/03/KHN-ICU-bed-county-analysis_2.zip | 2020-04-21 | 
| `kff-state-actions`| kff/raw_data.csv | https://www.kff.org/health-costs/issue-brief/state-data-and-policy-actions-to-address-coronavirus/#stateleveldata  | 2020-04-24  |
| `kff-state-actions`| kff/raw_data_0504.csv | https://www.kff.org/health-costs/issue-brief/state-data-and-policy-actions-to-address-coronavirus/#stateleveldata  | 2020-05-04  |
| `kff-state-health-policy-actions` | kff/health_policy_actions.csv | https://www.kff.org/health-costs/issue-brief/state-data-and-policy-actions-to-address-coronavirus/#stateleveldata | 2020-04-10 |
| `kff-state-actions-policy-date`| kff/response_policy_date.csv | https://www.kff.org/report-section/state-data-and-policy-actions-to-address-coronavirus-sources/  | 2020-04-24  |
| `wiki-state-regulations` | wiki/state_regulations | https://en.m.wikipedia.org/wiki/U.S.\_state_and_local_government_response_to_the_2020_coronavirus_pandemic | 2020-04-24 |
| `world-bank-hospital-beds`  | world-bank/API_SH.MED.BEDS.ZS_DS2_en_csv_v2_988924.csv | https://data.worldbank.org/indicator/SH.MED.BEDS.ZS  | 2020-04-24 |
| `world-bank-physicians` | world-bank/API_SH.MED.PHYS.ZS_DS2_en_csv_v2_993645.csv | https://data.worldbank.org/indicator/SH.MED.PHYS.ZS | 2020-04-24 | 
| `world-bank-nurses` | world-bank/API_SH.MED.NUMW.P3_DS2_en_csv_v2_993722.csv | https://data.worldbank.org/indicator/SH.MED.NUMW.P3 | 2020-04-24 | 
| `world-bank-specialist-surgical-workforce` | world-bank/API_SH.MED.SAOP.P5_DS2_en_csv_v2_993706.csv | https://data.worldbank.org/indicator/SH.MED.SAOP.P5 | 2020-04-24 | 
| `world-bank-elderly-population` | world-bank/API_SP.POP.65UP.TO.ZS_DS2_en_csv_v2_988979.csv | https://data.worldbank.org/indicator/SP.POP.65UP.TO.ZS?view=chart | 2020-04-24 | 
| `oxford-government-responses` | oxford/OxCGRT_Download_240420_164803_Full.csv | https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker | 2020-04-24 | 
| `jhu-global-confirmed` | jhu-csse/time_series_covid19_confirmed_global.csv | https://github.com/CSSEGISandData/COVID-19/blob/fef4e485ca4d46cd05deb5dc3a8e74e806bc9364/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv | 2020-04-24 | 
| `jhu-global-recovered` | jhu-csse/time_series_covid19_recovered_global.csv | https://github.com/CSSEGISandData/COVID-19/blob/fef4e485ca4d46cd05deb5dc3a8e74e806bc9364/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv | 2020-04-24 | 
| `jhu-global-death` | jhu-csse/time_series_covid19_deaths_global.csv | https://github.com/CSSEGISandData/COVID-19/blob/fef4e485ca4d46cd05deb5dc3a8e74e806bc9364/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv    | 2020-04-24 | 
| `population-density-state` | population-density/population_density_state-level.csv | https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population_density    | 2020-05-04 | 

We kept the original name of the datasets, and only performed necessary actions:

- All dataset from World Bank have metadata at the top (line 1-4). We removed those lines.
- `kff/raw_data.csv` includes some extra information in the spreadsheet (line 1-2, 56-71), which cause errors. We removed those lines from the table.We removed some lines from the `kff/raw_data_05-04.csv` and `kff/health_policy_actions.csv` for the same reason.

# Processing Country Level Data

At the country level, we are creating two tables `country_indicators.csv` and `country_responses.csv`. For the first table, we will collect the following information for each country:

| name  | meaning  | source  |
| :---- | :------- | :------ |
| `country_name` | the name of the country/region | `world-bank-hospital-beds` |
| `country_code` | the ISO 3 country code | `world-bank-hospital-beds` |
| `hospital_beds_per_1000` | number of hospital beds per 1000 people          |`world-bank-hospital-beds`     |
| `physicians_per_1000`   | number of physicians per 1000 people            |`world-bank-physicians`       |
| `nurses_per_1000`      | number of nurses and midwives per 1000 people      |`world-bank-nurses`          |
| `percentage_65up`      | population ages 65 and above (% of total population) |`world-bank-elderly-population`  |

In [2]:
# extract indicator SH.MED.BEDS.ZScountry_responses
hospital_beds_df = pd.read_csv('./data/world-bank/API_SH.MED.BEDS.ZS_DS2_en_csv_v2_988924.csv')

latest_numbers = []
for idx, row in hospital_beds_df.iterrows():
    latest = np.nan
    for y in range(2019,1959, -1):
        if pd.notnull(row[str(y)]):
            latest = row[str(y)]
            break
    latest_numbers.append(latest)

hospital_beds_df['hospital_beds_per_1000'] = pd.Series(latest_numbers)
hospital_beds = hospital_beds_df[['Country Name', 'hospital_beds_per_1000']]

hospital_beds.head(3)

,Country Name,hospital_beds_per_1000
0,Aruba,NaN
1,Afghanistan,0.5
2,Angola,0.8


In [3]:
# extract indicator SH.MED.PHYS.ZS
physicians_df = pd.read_csv('./data/world-bank/API_SH.MED.PHYS.ZS_DS2_en_csv_v2_993645.csv')

latest_numbers = []
for idx, row in physicians_df.iterrows():
    latest = np.nan
    for y in range(2019,1959, -1):
        if pd.notnull(row[str(y)]):
            latest = row[str(y)]
            break
    latest_numbers.append(latest)

physicians_df['physicians_per_1000'] = pd.Series(latest_numbers)
physicians = physicians_df[['Country Name', 'physicians_per_1000']]

physicians.head(3)

,Country Name,physicians_per_1000
0,Aruba,1.1200
1,Afghanistan,0.2840
2,Angola,0.2149


In [4]:
# extract indicator SH.MED.NUMW.P3
nurses_df = pd.read_csv('./data/world-bank/API_SH.MED.NUMW.P3_DS2_en_csv_v2_993722.csv')

latest_numbers = []
for idx, row in nurses_df.iterrows():
    latest = np.nan
    for y in range(2019,1959, -1):
        if pd.notnull(row[str(y)]):
            latest = row[str(y)]
            break
    latest_numbers.append(latest)

nurses_df['nurses_per_1000'] = pd.Series(latest_numbers)
nurses = nurses_df[['Country Name', 'nurses_per_1000']]

nurses.head(3)

,Country Name,nurses_per_1000
0,Aruba,NaN
1,Afghanistan,0.3200
2,Angola,1.3123


In [5]:
# extract indicator SP.POP.65UP.TO.ZS
elders_df = pd.read_csv('./data/world-bank/API_SP.POP.65UP.TO.ZS_DS2_en_csv_v2_988979.csv')

latest_numbers = []
for idx, row in elders_df.iterrows():
    latest = np.nan
    for y in range(2019,1959, -1):
        if pd.notnull(row[str(y)]):
            latest = row[str(y)]
            break
    latest_numbers.append(latest)

elders_df['percentage_65up'] = pd.Series(latest_numbers)
elders = elders_df[['Country Name', 'percentage_65up']]

elders.head(3)

,Country Name,percentage_65up
0,Aruba,13.550947
1,Afghanistan,2.584927
2,Angola,2.216374


In [6]:
# collect ISO 3 codes for each country
ISO3 = hospital_beds_df[['Country Name', 'Country Code']]
ISO3.head(3)

,Country Name,Country Code
0,Aruba,ABW
1,Afghanistan,AFG
2,Angola,AGO


After collecting all the pieces, we can now merge them and output to the output directory (`processed_data`).

In [7]:

# merge dataframes together by country name
data_frames = [ISO3, hospital_beds, physicians, nurses, elders]
df_merged = reduce(lambda left, right: pd.merge(left, right, on=['Country Name']), data_frames)

# rename columns names for consistency
df_merged.rename(columns={
    'Country Name': 'country_name', 
    'Country Code': 'country_code'
}, inplace=True)

df_merged.head(3)

,country_name,country_code,hospital_beds_per_1000,physicians_per_1000,nurses_per_1000,percentage_65up
0,Aruba,ABW,NaN,1.1200,NaN,13.550947
1,Afghanistan,AFG,0.5,0.2840,0.3200,2.584927
2,Angola,AGO,0.8,0.2149,1.3123,2.216374


In [8]:
# write to output file in processed_data folder
df_merged.to_csv('./processed_data/country_indicators.csv', index=False)

For the second table `country_responses.csv`, we will collect the timeseries data, each containing the following attributes:

| name | meaning | source |
| :--- | :------ | :----- |
| `date` | date of the data collected | `oxford-government-responses` |
| `country_name` | name of the country/region | `oxford-government-responses` |
| `country_code` | ISO 3 country code | `oxford-government-responses` |
| `s1_school_closing` | closing of schools/universities | `oxford-government-responses` |
| `s1_is_general` | whether `s1` is general or targeted | `oxford-government-responses` |
| `s2_workplace_closing` | closing of workplaces | `oxford-government-responses` |
| `s2_is_general` | whether `s2` is general or targeted | `oxford-government-responses` |
| `s3_cancel_public_events` | cancelling public events | `oxford-government-responses` |
| `s3_is_general` | whether `s3` is general or targeted | `oxford-government-responses` |
| `s4_close_public_transport` | closing public transport | `oxford-government-responses` |
| `s4_is_general` | whether `s4` is general or targeted | `oxford-government-responses` |
| `s5_public_info_campaigns` | public info campaign | `oxford-government-responses` |
| `s5_is_general` | whether `s5` is general or targeted | `oxford-government-responses` |
| `s6_restrictions_on_internal_movement` | restricting domestic travels | `oxford-government-responses` |
| `s6_is_general` | whether `s6` is general or targeted | `oxford-government-responses` |
| `s7_international_traval_controls` | restricting international travel | `oxford-government-responses` |
| `s12_testing_policies` | who can get tested | `oxford-government-responses` |
| `s13_contact_tracking` | tracking closely contacted people | `oxford-government-responses` |
| `stringency_index` | the sum of policy scores, measuring the strictness of the government policies | `oxford-government-responses` |
| `confirmed` | the number of confirmed cases | `oxford-government-responses` |
| `recovered` | the number of recovered cases | `jhu-global-recovered` |
| `deaths` | death toll | `oxford-government-responses` |


This table uses data from [Oxford COVID-19 Government Responses Tracker](https://www.bsg.ox.ac.uk/research/publications/variation-government-responses-covid-19). The authors use a novel index to measure the stringency of government responses. A total of 13 indicators (`S1`-`S13`) are used. Nine of them (the ones we chose) are non-financial indicators. A detailed explanation can be found [here](https://www.bsg.ox.ac.uk/sites/default/files/2020-04/BSG-WP-2020-031-v4.0_0.pdf). 

In [9]:
gov_responses_df = pd.read_csv('data/oxford/OxCGRT_Download_240420_164803_Full.csv')

interested_columns = [
    'Date', 'CountryName', 'CountryCode', 
    'S1_School closing', 'S1_IsGeneral',
    'S2_Workplace closing', 'S2_IsGeneral',
    'S3_Cancel public events', 'S3_IsGeneral',
    'S4_Close public transport', 'S4_IsGeneral',
    'S5_Public information campaigns', 'S5_IsGeneral',
    'S6_Restrictions on internal movement', 'S6_IsGeneral',
    'S7_International travel controls',
    'S12_Testing framework',
    'S13_Contact tracing',
    'StringencyIndex', 'ConfirmedCases', 'ConfirmedDeaths'
]

# select interested values
gov_responses = gov_responses_df[interested_columns]

gov_responses.head(3)

,Date,CountryName,CountryCode,S1_School closing,S1_IsGeneral,S2_Workplace closing,S2_IsGeneral,S3_Cancel public events,S3_IsGeneral,S4_Close public transport,...,S5_Public information campaigns,S5_IsGeneral,S6_Restrictions on internal movement,S6_IsGeneral,S7_International travel controls,S12_Testing framework,S13_Contact tracing,StringencyIndex,ConfirmedCases,ConfirmedDeaths
0,20200101,Aruba,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
1,20200102,Aruba,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
2,20200103,Aruba,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN


In [10]:
# extract recovered data
recovered_df = pd.read_csv('./data/jhu-csse/time_series_covid19_recovered_global.csv')

# ignore Province/State, Lat, and Long
recovered = recovered_df[['Country/Region'] + list(recovered_df.columns[4:])]

# convert to a long table by melting
recovered = recovered.melt(id_vars='Country/Region', var_name='date', value_name='recovered')

recovered.head(3)

,Country/Region,date,recovered
0,Afghanistan,1/22/20,0
1,Albania,1/22/20,0
2,Algeria,1/22/20,0


The JHU datasets does not have ISO 3 country code, which means we need to use country/region name to do the join. It is possible that different datasets use different names for the same country/region.

In [11]:
# country names in Oxford dataset (without dups)
countries1 = set(gov_responses['CountryName'])

# country names in JHU datasets (without dups)
countries2 = set(recovered['Country/Region'])

# names common to both dataset
common = countries1 & countries2

print('Names without matches\n')
print(f'Oxford dataset: {countries1 - common}\n')
print(f'JHU datasets: {countries2 - common}\n')

Names without matches

Oxford dataset: {'Myanmar', 'Taiwan', 'Lesotho', 'Greenland', 'Palestine', 'Slovak Republic', 'Macao', 'Aruba', 'Hong Kong', 'Guam', 'South Korea', 'Democratic Republic of Congo', 'Kyrgyz Republic', 'Puerto Rico', 'Cape Verde', 'Czech Republic', 'United States', 'Bermuda'}

JHU datasets: {'Equatorial Guinea', 'Eritrea', 'Western Sahara', 'Saint Vincent and the Grenadines', 'Holy See', 'Benin', 'Sao Tome and Principe', 'Fiji', 'Liberia', 'Suriname', 'Senegal', 'Monaco', 'Armenia', 'Latvia', 'Antigua and Barbuda', 'Somalia', 'Burma', 'Guinea-Bissau', 'Cabo Verde', 'Saint Kitts and Nevis', "Cote d'Ivoire", 'Congo (Kinshasa)', 'Liechtenstein', 'Malta', 'Bhutan', 'Yemen', 'Kyrgyzstan', 'North Macedonia', 'Haiti', 'Maldives', 'Montenegro', 'Timor-Leste', 'Saint Lucia', 'US', 'Congo (Brazzaville)', 'MS Zaandam', 'Georgia', 'Taiwan*', 'Belarus', 'Grenada', 'Central African Republic', 'Cambodia', 'Korea, South', 'Bahamas', 'Lithuania', 'Czechia', 'Slovakia', 'West Bank an

After inspecting the output, we need to perform some translations.

- "Slovakia" to "Slovak Republic"
- "Korea, South" to "South Korea"
- "Kyrgyzstan" to "Kyrgyz Republic"
- "Taiwan*" to "Taiwan"
- "Congo (Kinshasa)" to "Democratic Republic of Congo"
- "US" to "United States"
- "Czechia" to "Czech Republic"

In [12]:
def translate_country(countryName):
    if countryName == "Slovakia":
        return "Slovak Republic"
    elif countryName == "Korea, South":
        return "South Korea"
    elif countryName == "Kyrgyzstan":
        return "Kyrgyz Republic"
    elif countryName == "Taiwan*":
        return "Taiwan"
    elif countryName == "Congo (Kinshasa)":
        return "Democratic Republic of Congo"
    elif countryName == "US":
        return "United States"
    elif countryName == "Czechia":
        return "Czech Republic"
    else:
        return countryName

# translate country/region names in the JHU datasets
recovered['Country/Region'] = recovered['Country/Region'].map(lambda x: translate_country(x))

Similarly, the two datasets represent date using different format, we need to translate that too.

- `M/DD/YY` to `YYYYMMDD`

In [13]:
def translate_date(datestr):
    m, d, _ = datestr.split('/')
    return 2020 * 10000 + int(m) * 100 + int(d)

#translate country/region names in the JHU dataset
recovered['date'] = recovered['date'].map(lambda x: translate_date(x))

Now we can combine the dataframes by inner join and write the result to a CSV file.

In [14]:
# rename for joinning
recovered.rename(columns={
    'Country/Region': 'CountryName',
    'date': 'Date'
}, inplace=True)

# join dataframes on date and country name
df_merged = pd.merge(gov_responses, recovered, on=['Date', 'CountryName'])

# rename columns names for consistency
df_merged.rename(columns={
    'Date': 'date',
    'CountryName': 'country_name',
    'CountryCode': 'country_code',
    'S1_School closing': 's1_school_closing',
    'S1_IsGeneral': 's1_is_general',
    'S2_Workplace closing': 's2_workplace_closing',
    'S2_IsGeneral': 's2_is_general',
    'S3_Cancel public events': 's3_cancel_public_events',
    'S3_IsGeneral': 's3_is_general',
    'S4_Close public transport': 's4_close_public_transport',
    'S4_IsGeneral': 's4_is_general',
    'S5_Public information campaigns': 's5_public_information_campaigns',
    'S5_IsGeneral': 's5_is_general',
    'S6_Restrictions on internal movement': 's6_restrictions_on_internal_movement',
    'S6_IsGeneral': 's6_is_general',
    'S7_International travel controls': 's7_international_travel_controls',
    'S12_Testing framework': 's12_testing_framework',
    'S13_Contact tracing': 's13_contact_tracing',
    'StringencyIndex': 'stringency_index',
    'ConfirmedCases': 'confirmed',
    'ConfirmedDeaths': 'deaths'
}, inplace=True)

df_merged.head(3)

,date,country_name,country_code,s1_school_closing,s1_is_general,s2_workplace_closing,s2_is_general,s3_cancel_public_events,s3_is_general,s4_close_public_transport,...,s5_is_general,s6_restrictions_on_internal_movement,s6_is_general,s7_international_travel_controls,s12_testing_framework,s13_contact_tracing,stringency_index,confirmed,deaths,recovered
0,20200122,Afghanistan,AFG,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
1,20200123,Afghanistan,AFG,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
2,20200124,Afghanistan,AFG,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0


In [15]:
# write to output file in processed_data folder
df_merged.to_csv('./processed_data/country_responses.csv', index=False)

## Processing State Level Data

At the state level, we are creating for tables `state_indicators.csv`, `state_responses_04-24.csv`, `state_responses_05-04.csv` and `state_cases.csv`. For the first table, we will collect the following information for each state: 

| name  | meaning  | source  |
| :---- | :------- | :------ |
| `state_name` | the name of the state | `khn-icu-beds-by-county` |
| `state_code` | the code of the state | `khn-hospital-by-county` |
| `hospitals_per_1000`      | the number of hospitals per 1000 people |`khn-hospital-by-county`  |
| `icu_beds_per_1000` | number of icu beds per 1000 people | `khn-icu-beds-by-county` |
| `percentage_60up`      | population ages 60 and above (% of total population) |`khn-icu-beds-by-county`  |
| `population_density_km2`      | population density of state |`population-density-state`  |
| `state_population`      | population of state |`khn-hospital-by-county`  |

In [16]:
#collect the number of icu beds per 1000
icu_beds_df = pd.read_csv('./data/khn/data-FPBfZ.csv')
icu_beds_df = icu_beds_df[['State', 'ICU Beds','Total Population']]
icu_beds = icu_beds_df.groupby(['State'], as_index=False).sum()

icu_beds['ICU Beds'] = 1000 * icu_beds['ICU Beds']/icu_beds['Total Population']
icu_beds = icu_beds[['State', 'ICU Beds']]

icu_beds.rename(columns={
    'State': 'state_name', 
    'ICU Beds': 'icu_beds_per_1000'
}, inplace=True)

icu_beds.head(3)

,state_name,icu_beds_per_1000
0,Alabama,0.316032
1,Alaska,0.161123
2,Arizona,0.228930


In [17]:
#collect the percentage of people ages 60 and above
elders_df = pd.read_csv('./data/khn/data-FPBfZ.csv')
elders_df = elders_df[['State', 'Population Aged 60+', 'Total Population']]
elders = elders_df.groupby(['State'], as_index=False).sum()

elders['Population Aged 60+'] = 100*elders['Population Aged 60+']/elders['Total Population']

elders = elders.rename(columns={
    'State': 'state_name', 
    'Population Aged 60+': 'percentage_60up',
    'Total Population': 'state_population'
})

elders.head(3)

,state_name,percentage_60up,state_population
0,Alabama,21.968157,4850771
1,Alaska,15.847894,738565
2,Arizona,22.066078,6809946


In [18]:
#collect the population density of state
pop_density_df = pd.read_csv('./data/population-density/population_density_state-level.csv')

pop_density_df = pop_density_df[['State etc.', 'Population density(per km2)']]


pop_density = pop_density_df.rename(columns={
    'State etc.': 'state_name', 
    'Population density(per km2)': 'population_density_km2'
})
pop_density.head(3)

,state_name,population_density_km2
0,Alabama,37
1,Alaska,0
2,American Samoa,279


In [19]:
#collect the hospitals number in cost reports
hospitals_in_df = pd.read_csv('./data/khn/KHN_ICU_bed_county_analysis_2.csv')

hospitals_in_cost_df = hospitals_in_df[['state', 'st', 'hospitals_in_cost_reports', 'Total_pop']]
hospitals_in_cost = hospitals_in_cost_df.groupby(['state','st'], as_index=False).sum()


hospitals_in_cost['hospitals_in_cost_reports'] = 1000*hospitals_in_cost['hospitals_in_cost_reports']/hospitals_in_cost['Total_pop']
hospitals_in_cost = hospitals_in_cost[['state', 'st', 'hospitals_in_cost_reports']]

hospitals_in_cost = hospitals_in_cost.rename(columns={
    'state': 'state_name', 
    'st': 'state_code',
    'hospitals_in_cost_reports': 'hospitals_per_1000'
})

hospitals_in_cost.head(3)

,state_name,state_code,hospitals_per_1000
0,Alabama,AL,0.017523
1,Alaska,AK,0.029787
2,Arizona,AZ,0.010720


In [20]:
# merge dataframes together by state name
data_frames = [hospitals_in_cost, icu_beds, elders, pop_density]
df_merged = reduce(lambda left, right: pd.merge(left, right, how='inner', on=['state_name']), data_frames)


df_merged.head(3)

,state_name,state_code,hospitals_per_1000,icu_beds_per_1000,percentage_60up,state_population,population_density_km2
0,Alabama,AL,0.017523,0.316032,21.968157,4850771,37
1,Alaska,AK,0.029787,0.161123,15.847894,738565,0
2,Arizona,AZ,0.010720,0.228930,22.066078,6809946,23


In [21]:
# write to output file in processed_data folder
df_merged.to_csv('./processed_data/state_indicators.csv', index=False)


For the second table `state_responses_04-24.csv`, we will collect the following information for each state:

| name  | meaning  | source  |
| :---- | :------- | :------ |
| `state_name` | the name of the state | `kff-state-actions` |
| `state_is_easing_social_distancing_measures` | state is easing social distancing measures | `kff-state-actions` |
| `stay_at_home_order` | order scope         |`kff-state-actions`     |
| `date_when_stay_at_home_ordered`      | date when stay at home ordered |`wiki-state-regulations`  |
| `mandatory_quarantine_for_travelers`   | mandatory quarantine for travelers           |`kff-state-actions`       |
| `non-essential_business_closures`      | business closures      |`kff-state-actions`          |
| `large_gatherings_ban`      | gatherings scope ban     |`kff-state-actions`          |
| `school_closures`      | details about school closures |`kff-state-actions`  |
| `bar/restaurant_limits`      | detail about bar/resurant limits |`kff-state-actions`  |
| `primary_election_postponement`      | whether primary election is postponed |`kff-state-actions`  |
| `emergency_declaration`      | emergency declaration |`kff-state-actions`  |
| `date_of_state_emergency_declared`      | date of state emergency declared |`wiki-state-regulations`  |
| `waive_cost_sharing_for_COVID-19_treatment`      | waive cost sharing for COVID-19 treatment |`kff-state-health-policy-actions`  |
| `free_cost_vaccine_when_available`      | free cost vaccine when available |`kff-state-health-policy-actions`  |
| `state_requires_waiver_of_prior_authorization_requirements`      | state requires waiver of prior authorization requirements |`kff-state-health-policy-actions`  |
| `early_Prescription_Refills`      | early Prescription Refills |`kff-state-health-policy-actions`  |
| `premium_payment_grace period`      | premium payment grace period |`kff-state-health-policy-actions`  |
| `marketplace_special_enrollment_period(SEP)`      | marketplace special enrollment period (SEP) |`kff-state-health-policy-actions`  |
| `section_1135_waiver`      | if waiver is approved |`kff-state-health-policy-actions`  |
| `paid_sick_leave`      | paid sick leave |`kff-state-health-policy-actions`  |
| `daycares`      | daycares |`wiki-state-regulations`  |
| `public_health_emergency_date`      | date when public health emergency declare |`kff-state-actions-policy-date`  |
| `school_closures_date`      | date when school closure declare|`kff-state-actions-policy-date`  |
| `large_gatherings_ban_date`      | date when announce large gathering ban declare|`kff-state-actions-policy-date`  |
| `bar/restaurant_limits_date`      | date when announce bar/restaurant limits declare|`kff-state-actions-policy-date`  |
| `primary_election_postponement_date`      | date when announce primary election postponement |`kff-state-actions-policy-date`  |
| `section_1135_waiver_date`      | date when section 1135 waiver declare |`kff-state-actions-policy-date`  |
| `non-essential_business_closures_date`      | date when non-essentail bussinuess closures declare |`kff-state-actions-policy-date`  |
| `stay_at_home_order_date`      | date when stay at home order in effect |`kff-state-actions-policy-date`  |
| `early_rx_refill_date`      | date when early rx refill declare |`kff-state-actions-policy-date`  |
| `mandatory_quarantine_date`      | date when mandatory quarantine declare |`kff-state-actions-policy-date`  |
| `premium_grace_period_date`      | date when premium grace period declare |`kff-state-actions-policy-date`  |
| `emergency_declaration_date`      | date when emergency declare |`kff-state-actions-policy-date`  |
| `marketplace_sep_date`      | date when marketplace sep declare|`kff-state-actions-policy-date`  |
| `waive_prior_auth_date`      | date when waive prior auth decla |`kff-state-actions-policy-date`  |




For the third table `state_responses_05-04.csv`, its column it is almost the same as 'state_responses_04-25.csv'.
It does not have date columns for reducing duplicate values.

In [22]:
state_actions_df=pd.read_csv('./data/kff/raw_data.csv')
state_actions=state_actions_df[list(state_actions_df.columns[:-1])]
#remove national total result record
state_actions=state_actions.drop(0)
state_actions=state_actions.reset_index(drop=True)
state_actions.head(3)

,Location,State Is Easing Social Distancing Measures,Stay At Home Order,Mandatory Quarantine for Travelers,Non-Essential Business Closures,Large Gatherings Ban,School Closures,Bar/Restaurant Limits,Primary Election Postponement,Emergency Declaration
0,Alabama,-,Statewide,-,All Non-Essential Businesses,>10 People Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,Yes,Yes
1,Alaska,-,Statewide,All Travelers,All Non-Essential Businesses,All Gatherings Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,-,Yes
2,Arizona,-,Statewide,From Certain States,All Non-Essential Businesses,>10 People Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,-,Yes


In [23]:
state_health_policy_actions_df=pd.read_csv('./data/kff/health_policy_actions.csv')
state_health_policy_actions=state_health_policy_actions_df[list(state_health_policy_actions_df.columns[:-1])]
#remove national total result record
state_health_policy_actions=state_health_policy_actions.drop(0)
state_health_policy_actions=state_health_policy_actions.reset_index(drop=True)
state_health_policy_actions.head(3)

,Location,Waive Cost Sharing for COVID-19 Treatment,Free Cost Vaccine When Available,State Requires Waiver of Prior Authorization Requirements*,Early Prescription Refills,Premium Payment Grace Period,Marketplace Special Enrollment Period (SEP),Section 1135 Waiver,Paid Sick Leave
0,Alabama,-,-,-,-,-,-,Approved,-
1,Alaska,-,-,-,State Requires,All Policies,-,Approved,-
2,Arizona,-,-,-,-,-,-,Approved,Enacted


In [24]:
state_regulations_df=pd.read_csv('./data/wiki/state_regulations.csv',delimiter="\t")
state_regulations_df.head(3)

,State/territory,State of emergency declared,Stay at home ordered,Gatherings banned,Out-of-state travel restrictions,Schools,Daycares,Bars & sit-down restaurants,Non-essential retail,Sources
0,Alabama?Alabama,13-Mar,4-Apr,10 or more,No,Yes (remainder of term),Yes,Yes,Yes,[4][5][6][7][8][9]
1,Alaska?Alaska,11-Mar,28-Mar,10 or more,Mandatory quarantine,Yes (remainder of term),Yes,Yes,Yes,[10][6][11][7][8]
2,American Samoa?American Samoa,29-Jan,No,10 or more,Travel suspended,Yes (until further notice),Yes,No,No,[12][13][14]


There are similar columns between wiki-state-regulations and kff-state-actions:

- "Gatherings banned" and "Large Gatherings Ban"
- "Out-of-state travel restrictions" and "Mandatory Quarantine for Travelers"
- "Schools" and "School Closures"
- "Bars & sit-down restaurants" and "Bar/Restaurant Limits"
- "Non-essential retail" and "Non-Essential Business Closures"

So for similar columns we only choose columns from kff-state-actions.

In [25]:
def clean_state_name(name):
    return name.split('?')[0]

state_regulations_df['State/territory']=state_regulations_df['State/territory'].map(lambda x:clean_state_name(x))
state_regulations=state_regulations_df[list(state_regulations_df.columns[:3])+['Daycares']]
state_regulations.head(3)

,State/territory,State of emergency declared,Stay at home ordered,Daycares
0,Alabama,13-Mar,4-Apr,Yes
1,Alaska,11-Mar,28-Mar,Yes
2,American Samoa,29-Jan,No,Yes


We need to use state names to do the join. It is possible that different datasets use different names for the same location.

In [26]:
# state names in kff dataset 
states1 = set(state_actions['Location'])

# country names in wiki datasets 
states2 = set(state_regulations['State/territory'])

# names common to both dataset
common_states = states1 & states2

print('Names with no matches: \n')
print(f'KFF dataset: {states1-common_states}\n')
print(f'Wiki dataset: {states2-common_states}\n')

Names with no matches: 

KFF dataset: {'District of Columbia', 'New York', 'Georgia', 'Washington'}

Wiki dataset: {'Washington, D.C.', 'Northern Mariana Islands', 'Guam', 'United States Virgin Islands', 'Puerto Rico', 'Washington (state)', 'American Samoa', 'New York (state)', 'Georgia (U.S. state)'}



After inspecting the output, so we need to perform some translations.
- "Washington, D.C." to "District of Columbia"
- "Washington (state)" to "Washington"
- "New York (state)" to "New York"
- "Georgia (U.S. state)" to "Georgia"

KFF dataset does not have the territory record, so the following territory records are removed.
- Puerto Rico
- Northern Mariana Islands
- Guam
- United States Virgin Islands
- American Samoa

We then rename the column names for joining:
- change Wiki dataset column `State/territory` to `state_name`
- change KFF datasets column `Location` to `state_name`


In [27]:
def translate_state(stateName):
    if stateName == "Washington, D.C.":
        return "District of Columbia"
    elif stateName == "Washington (state)":
        return "Washington"
    elif stateName == "New York (state)":
        return "New York"
    elif stateName == "Georgia (U.S. state)":
        return "Georgia"
    else:
        return stateName


#rename column names for joining
state_actions.rename(columns={'Location':'state_name'}, inplace=True) 
state_health_policy_actions.rename(columns={'Location':'state_name'}, inplace=True) 
state_regulations.rename(columns={'State/territory':'state_name'}, inplace=True) 

#remove territory records
state_regulations=state_regulations.drop(state_regulations[state_regulations.state_name=="Puerto Rico"].index)
state_regulations=state_regulations.drop(state_regulations[state_regulations.state_name=="Northern Mariana Islands"].index)
state_regulations=state_regulations.drop(state_regulations[state_regulations.state_name=="Guam"].index)
state_regulations=state_regulations.drop(state_regulations[state_regulations.state_name=="United States Virgin Islands"].index)
state_regulations=state_regulations.drop(state_regulations[state_regulations.state_name=="American Samoa"].index)
state_regulations=state_regulations.reset_index(drop=True)

# translate state names in the wiki dataset
state_regulations['state_name'] = state_regulations['state_name'].map(lambda x: translate_state(x))
 
state_regulations.head(3)

,state_name,State of emergency declared,Stay at home ordered,Daycares
0,Alabama,13-Mar,4-Apr,Yes
1,Alaska,11-Mar,28-Mar,Yes
2,Arizona,11-Mar,31-Mar,Yes


In [28]:
date_df=pd.read_csv('./data/kff/response_policy_date.csv')
date_df.fillna('', inplace=True)
date_df.head(3)

,state_name,public_health_emergency_date,school_closures_date,large_gatherings_ban_date,bar/restaurant_limits_date,primary_election_postponement_date,section_1135_waiver_date,non-essential_business_closures_date,stay_at_home_order_date,early_rx_refill_date,mandatory_quarantine_date,premium_grace_period_date,emergency_declaration_date,marketplace_sep_date,waive_prior_auth_date
0,Alabama,3/13,3/17,3/17,3/17,3/18,3/23,3/27,3/27,,,,,,
1,Alaska,3/11,3/13,3/20,3/17,,4/2,3/20,3/17,3/3,3/17,3/18,,,
2,Arizona,3/11,3/16,3/30,3/19,,3/23,3/30,3/30,,4/7,,3/11,,


Now we combine four dateframes into one dateframe.

In [29]:
# perform inner join of four tables
state_merged = pd.merge(state_actions, state_health_policy_actions, on=['state_name'])
state_merged = pd.merge(state_merged, state_regulations, on=['state_name'])
state_merged=pd.merge(state_merged,date_df,on=['state_name'])
#change column name
state_merged.columns=state_merged.columns.map(lambda x:x.lower())
state_merged.rename(columns={
    'marketplace special enrollment period (sep)':'marketplace special enrollment period(SEP)',
    'state of emergency declared':'date of state emergency declared',
    'stay at home ordered':'date when stay at home ordered',
    'state requires waiver of prior authorization requirements*':'state requires waiver of prior authorization requirements'
}, inplace=True)
state_merged.head(3)

,state_name,state is easing social distancing measures,stay at home order,mandatory quarantine for travelers,non-essential business closures,large gatherings ban,school closures,bar/restaurant limits,primary election postponement,emergency declaration,...,primary_election_postponement_date,section_1135_waiver_date,non-essential_business_closures_date,stay_at_home_order_date,early_rx_refill_date,mandatory_quarantine_date,premium_grace_period_date,emergency_declaration_date,marketplace_sep_date,waive_prior_auth_date
0,Alabama,-,Statewide,-,All Non-Essential Businesses,>10 People Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,Yes,Yes,...,3/18,3/23,3/27,3/27,,,,,,
1,Alaska,-,Statewide,All Travelers,All Non-Essential Businesses,All Gatherings Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,-,Yes,...,,4/2,3/20,3/17,3/3,3/17,3/18,,,
2,Arizona,-,Statewide,From Certain States,All Non-Essential Businesses,>10 People Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,-,Yes,...,,3/23,3/30,3/30,,4/7,,3/11,,


In [30]:
def name_form(column_name):
    lists = column_name.split(" ")
    return '_'.join(lists)

state_merged.columns=state_merged.columns.map(lambda x:name_form(x))

#change column sequence
df_date1=state_merged.date_when_stay_at_home_ordered
df_date2=state_merged.date_of_state_emergency_declared
state_merged=state_merged.drop('date_when_stay_at_home_ordered',axis=1)
state_merged=state_merged.drop('date_of_state_emergency_declared',axis=1)
state_merged.insert(3,'date_when_stay_at_home_ordered',df_date1)
state_merged.insert(11,'date_of_state_emergency_declared',df_date2)

#replace '-' with empty string
for col in state_merged.columns:
    state_merged[col] = state_merged[col].map(lambda x: '' if x == '-' else x)
    

state_merged.head(3)

,state_name,state_is_easing_social_distancing_measures,stay_at_home_order,date_when_stay_at_home_ordered,mandatory_quarantine_for_travelers,non-essential_business_closures,large_gatherings_ban,school_closures,bar/restaurant_limits,primary_election_postponement,...,primary_election_postponement_date,section_1135_waiver_date,non-essential_business_closures_date,stay_at_home_order_date,early_rx_refill_date,mandatory_quarantine_date,premium_grace_period_date,emergency_declaration_date,marketplace_sep_date,waive_prior_auth_date
0,Alabama,,Statewide,4-Apr,,All Non-Essential Businesses,>10 People Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,Yes,...,3/18,3/23,3/27,3/27,,,,,,
1,Alaska,,Statewide,28-Mar,All Travelers,All Non-Essential Businesses,All Gatherings Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,,...,,4/2,3/20,3/17,3/3,3/17,3/18,,,
2,Arizona,,Statewide,31-Mar,From Certain States,All Non-Essential Businesses,>10 People Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,,...,,3/23,3/30,3/30,,4/7,,3/11,,


In [31]:
# write to output file in processed_data folder
state_merged.to_csv('./processed_data/state_responses_04-24.csv', index=False)

For `state_responses_05-05` we do similar operations:

In [32]:
state_actions_df2=pd.read_csv('./data/kff/raw_data_05-04.csv')
state_actions2=state_actions_df2[list(state_actions_df2.columns[:-1])]

#remove national total result record
state_actions2=state_actions2.drop(0)
state_actions2=state_actions2.reset_index(drop=True)
state_actions2.rename(columns={'Location':'state_name'}, inplace=True) 


# perform inner join of three tables
state_merged2 = pd.merge(state_actions2, state_health_policy_actions, on=['state_name'])
state_merged2 = pd.merge(state_merged2, state_regulations, on=['state_name'])
#change column name
state_merged2.columns=state_merged2.columns.map(lambda x:x.lower())
state_merged2.rename(columns={
    'marketplace special enrollment period (sep)':'marketplace special enrollment period(SEP)',
    'state of emergency declared':'date of state emergency declared',
    'stay at home ordered':'date when stay at home ordered',
    'state requires waiver of prior authorization requirements*':'state requires waiver of prior authorization requirements'
}, inplace=True)

def name_form(column_name):
    lists = column_name.split(" ")
    return '_'.join(lists)

state_merged2.columns=state_merged2.columns.map(lambda x:name_form(x))

#change column sequence
df_date3=state_merged2.date_when_stay_at_home_ordered
df_date4=state_merged2.date_of_state_emergency_declared
state_merged2=state_merged2.drop('date_when_stay_at_home_ordered',axis=1)
state_merged2=state_merged2.drop('date_of_state_emergency_declared',axis=1)
state_merged2.insert(3,'date_when_stay_at_home_ordered',df_date3)
state_merged2.insert(11,'date_of_state_emergency_declared',df_date4)

#replace '-' with empty string
for col in state_merged2.columns:
    state_merged2[col] = state_merged2[col].map(lambda x: '' if x == '-' else x)
    
# write to output file in processed_data folder
state_merged2.to_csv('./processed_data/state_responses_05-04.csv', index=False)
state_merged2.head(3)


,state_name,state_is_easing_social_distancing_measures,stay_at_home_order,date_when_stay_at_home_ordered,mandatory_quarantine_for_travelers,non-essential_business_closures,large_gatherings_ban,school_closures,restaurant_limits,primary_election_postponement,...,date_of_state_emergency_declared,waive_cost_sharing_for_covid-19_treatment,free_cost_vaccine_when_available,state_requires_waiver_of_prior_authorization_requirements,early_prescription_refills,premium_payment_grace_period,marketplace_special_enrollment_period(SEP),section_1135_waiver,paid_sick_leave,daycares
0,Alabama,Yes,Lifted,4-Apr,,Some Non-Essential Businesses Permitted to Reo...,>10 People Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,,...,13-Mar,,,,,,,Approved,,Yes
1,Alaska,Yes,Lifted,28-Mar,All Travelers,Some Non-Essential Businesses Permitted to Reo...,Expanded to 20+ People Prohibited,Closed for School Year,Reopened to Dine-in Service with Capacity Limits,NaN,...,11-Mar,,,,State Requires,All Policies,,Approved,,Yes
2,Arizona,,Statewide,31-Mar,From Certain States,All Non-Essential Businesses Closed,>10 People Prohibited,Closed for School Year,Closed Except for Takeout/Delivery,,...,11-Mar,,,,,,,Approved,Enacted,Yes


For the fourth table `state_cases.csv`, we will collect the timeseries data, each containing the following attributes:

| name  | meaning  | source  |
| :---- | :------- | :------ |
| `date` | date | `covid-tracking-states-daily` |
| `state_name` | state name | `covid-tracking-states-daily` |
| `totaltestresults` | the number of test results| `covid-tracking-states-daily` |
| `confirmed` | the number of confirmed cases | `covid-tracking-states-daily` |
| `recovered` | the number of recovered cases | `covid-tracking-states-daily` |
| `deaths` | death toll | `covid-tracking-states-daily` |

In [33]:
state_cases_df=pd.read_csv('./data/covid-tracking/states_daily_4pm_et.csv')
state_cases=state_cases_df[['date','state','totalTestResults','positive','recovered','death']]
state_cases.head(3)

,date,state,totalTestResults,positive,recovered,death
0,20200423,AK,12161.0,337.0,209.0,9.0
1,20200423,AL,52641.0,5778.0,NaN,197.0
2,20200423,AR,31590.0,2465.0,902.0,45.0


In [34]:
#change column name
state_cases.columns=state_cases.columns.map(lambda x:x.lower())

state_cases.rename(columns={
    'state':'state_name',
    'positive':'confirmed',
    'death':'deaths'
}, inplace=True)

state_cases.head(3)


,date,state_name,totaltestresults,confirmed,recovered,deaths
0,20200423,AK,12161.0,337.0,209.0,9.0
1,20200423,AL,52641.0,5778.0,NaN,197.0
2,20200423,AR,31590.0,2465.0,902.0,45.0


In [35]:
#change state code to state name
statecode = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

state_cases.state_name=state_cases.state_name.map(lambda x: statecode[x])
state_cases.head(3)

,date,state_name,totaltestresults,confirmed,recovered,deaths
0,20200423,Alaska,12161.0,337.0,209.0,9.0
1,20200423,Alabama,52641.0,5778.0,NaN,197.0
2,20200423,Arkansas,31590.0,2465.0,902.0,45.0


Check if state names in state_cases and state_actions are the same.

In [36]:
# state names in kff dataset 
states1 = set(state_actions['state_name'])

# country names in covid-tracking datasets 
states2 = set(state_cases['state_name'])

# names common to both dataset
common_states = states1 & states2

print('names with no matches: \n')
print(f'KFF dataset: {states1-common_states}\n')
print(f'COVID-tracking dataset: {states2-common_states}\n')

names with no matches: 

KFF dataset: set()

COVID-tracking dataset: {'Northern Mariana Islands', 'Virgin Islands', 'Guam', 'Puerto Rico', 'American Samoa'}



After inspecting the output, we see that KFF dataset does not have the territory record. Therefore, the following records are removed.

- Puerto Rico
- Northern Mariana Islands
- Guam
- Virgin Islands
- American Samoa


In [37]:
#remove territory records
state_cases=state_cases.drop(state_cases[state_cases.state_name=="Puerto Rico"].index)
state_cases=state_cases.drop(state_cases[state_cases.state_name=="Northern Mariana Islands"].index)
state_cases=state_cases.drop(state_cases[state_cases.state_name=="Guam"].index)
state_cases=state_cases.drop(state_cases[state_cases.state_name=="Virgin Islands"].index)
state_cases=state_cases.drop(state_cases[state_cases.state_name=="American Samoa"].index)
state_cases=state_cases.reset_index(drop=True)

state_cases.head(3)

,date,state_name,totaltestresults,confirmed,recovered,deaths
0,20200423,Alaska,12161.0,337.0,209.0,9.0
1,20200423,Alabama,52641.0,5778.0,NaN,197.0
2,20200423,Arkansas,31590.0,2465.0,902.0,45.0


In [38]:
# write to output file in processed_data folder
state_cases.to_csv('./processed_data/state_cases.csv', index=False)

## Calculating State Government Response Stringency Index


Based on [Oxford Variation in government responses to COVID-19](https://www.bsg.ox.ac.uk/sites/default/files/2020-05/BSG-WP-2020-032-v5.0_0.pdf), we use similar algorithm to calculate state government response stringency index.

First, classify the columns in the `state_responses_04-24.csv`.

| ID  | Name  |instructions|
| :---- | :------- |:------- |
|`Containment and closure`||
| `C1` | school_closures |0- No Actions<br>1-Recommended Closure for School Year or Recommended Closure<br>2-Closed or Closed for School Year|
| `C2` | non-essential_business_closures |0- No Actions<br>1- Certain Non-Essential Businesses<br>3- All Non-Essential Businesses |
| `C3`      | large_gatherings_ban |0- No Actions<br>1- Other<br>3- >10 People Prohibited<br>4- All Gatherings Prohibited|
| `C4` | stay_at_home_order|0- No Actions<br>1- High-risk Groups or Other<br>2- Statewide|
| `C5`      | mandatory_quarantine_for_travelers|0- No Actions<br>1- From Certain States or ALL Air Travelers or Other<br>2- All Travelers|
| `C6`      | bar/restaurant_limits|0- No Actions<br>1- Other<br>2- Limited On-Site Service <br>3- Closed Except for Takeout/Delivery|
| `C7`      | primary_election_postponement|0- No Actions<br>1- Yes|
| `C8`      |daycares|0- No<br>1- Restrict<br> 2- Yes |
| `Heatlh systems`      | ||
| `H1`      | waive_cost_sharing_for_covid-19_treatment|0- No Actions<br>1- State-Insurer Agreement<br>2- State Requires|
| `H2`      | free_cost_vaccine_when_available|0- No Actions<br>1- State-Insurer Agreement<br>2- State Requires|
| `H3`      | state_requires_waiver_of_prior_authorization_requirements|0- No Actions<br>1- For COVID-19 Testing<br>2- For COVID-19 Testing and Treatment|
| `H4`      | early_prescription_refills|0- No Actions<br>1- State Requires|
| `H5`      | premium_payment_grace_period|0- No Actions<br>1- COVID-19 Diagnosis/Impacts Only<br>2- All Policies|
| `H6`      | marketplace_special_enrollment_period(SEP)|0- No Actions<br>1- Yes |
| `H7`      |section_1135_waiver|0- No Actions<br>1- Approved|
| `H8`      |paid_sick_leave|0- No Actions<br>1- Proposed<br> 2- Enacted|
| `Other`      | ||
| `O1`      | state_is_easing_social_distancing_measures|0- No Actions<br>-1- Have Actions|







Then classify the columns in the `state_responses_05-04.csv`.

| ID  | Name  |instructions|
| :---- | :------- |:------- |
|`Containment and closure`||
| `C1` | school_closures |0- No Actions<br>1-Recommended Closure for School Year or Recommended Closure<br>2-Closed or Closed for School Year|
| `C2` | non-essential_business_closures |0- No Actions<br>1- Some Non-Essential Businesses Closed or All Non-Essential Businesses Permitted to Reopen with Reduced Capacity or Some Non-Essential Businesses Closed<br>2- Some Non-Essential Businesses Permitted to Reopen with Reduced Capacity<br>3- All Non-Essential Businesses Closed |
| `C3`      | large_gatherings_ban |0- No Actions or Lifted<br>1- Expanded to >25 People Prohibited or Expanded to 25+ People Prohibited or Other<br>2-  Expanded to 20+ People Prohibited  <br>3- Expanded to >10 People Prohibited or >10 People Prohibited<br>4- All Gatherings Prohibited|
| `C4` | stay_at_home_order|0- No Actions or Lifted<br>1- High-risk Groups or Rolled Back to High Risk Groups<br>2- Statewide|
| `C5`      | mandatory_quarantine_for_travelers|0- No Actions or Lifted<br>1- From Certain States or ALL Air Travelers or Rolled Back to Certain States<br>2- All Travelers|
| `C6`      | restaurant_limits|0- No Actions<br>1- Reopened to Dine-in Service<br>2- Reopened to Dine-in Service with Capacity Limits or Limited Dine-in Service<br>3- Closed Except for Takeout/Delivery|
| `C7`      | primary_election_postponement|0- No Actions<br>1- Yes<br>2- Canceled|
| `C8`      |daycares|0- No<br>1- Restrict<br> 2- Yes |
|   `Heatlh systems`    | ||
| `H1`      | waive_cost_sharing_for_covid-19_treatment|0- No Actions<br>1- State-Insurer Agreement<br>2- State Requires|
| `H2`      | free_cost_vaccine_when_available|0- No Actions<br>1- State-Insurer Agreement<br>2- State Requires|
| `H3`      | state_requires_waiver_of_prior_authorization_requirements|0- No Actions<br>1- For COVID-19 Testing<br>2- For COVID-19 Testing and Treatment|
| `H4`      | early_prescription_refills|0- No Actions<br>1- State Requires|
| `H5`      | premium_payment_grace_period|0- No Actions<br>1- COVID-19 Diagnosis/Impacts Only<br>2- All Policies|
| `H6`      | marketplace_special_enrollment_period(SEP)|0- No Actions<br>1- Yes |
| `H7`      |section_1135_waiver|0- No Actions<br>1- Approved|
| `H8`      |paid_sick_leave|0- No Actions<br>1- Proposed<br> 2- Enacted|
| `Other`      | ||
| `O1`      | state_is_easing_social_distancing_measures|0- No Actions<br>-1- Yes|


Calculate state government response stringency index on 2020/04/24.

In [39]:
state_index_df=pd.read_csv('./processed_data/state_responses_04-24.csv')
state_index=state_index_df[['state_name','school_closures','non-essential_business_closures','large_gatherings_ban','stay_at_home_order',
                            'mandatory_quarantine_for_travelers','bar/restaurant_limits','primary_election_postponement',
                           'daycares','waive_cost_sharing_for_covid-19_treatment','free_cost_vaccine_when_available','state_requires_waiver_of_prior_authorization_requirements',
                           'early_prescription_refills','premium_payment_grace_period','marketplace_special_enrollment_period(SEP)','section_1135_waiver','paid_sick_leave'
                            ,'state_is_easing_social_distancing_measures']]

state_index.head(3)

,state_name,school_closures,non-essential_business_closures,large_gatherings_ban,stay_at_home_order,mandatory_quarantine_for_travelers,bar/restaurant_limits,primary_election_postponement,daycares,waive_cost_sharing_for_covid-19_treatment,free_cost_vaccine_when_available,state_requires_waiver_of_prior_authorization_requirements,early_prescription_refills,premium_payment_grace_period,marketplace_special_enrollment_period(SEP),section_1135_waiver,paid_sick_leave,state_is_easing_social_distancing_measures
0,Alabama,Closed for School Year,All Non-Essential Businesses,>10 People Prohibited,Statewide,NaN,Closed Except for Takeout/Delivery,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Approved,NaN,NaN
1,Alaska,Closed for School Year,All Non-Essential Businesses,All Gatherings Prohibited,Statewide,All Travelers,Closed Except for Takeout/Delivery,NaN,Yes,NaN,NaN,NaN,State Requires,All Policies,NaN,Approved,NaN,NaN
2,Arizona,Closed for School Year,All Non-Essential Businesses,>10 People Prohibited,Statewide,From Certain States,Closed Except for Takeout/Delivery,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Approved,Enacted,NaN


In [40]:
def C1(val):
    val=str(val)
    if 'Recommended' in val:
        return 1
    if 'Closed' in val:
        return 2
    return 0

def C2(val):
    val=str(val)
    if 'Certain Non-Essential Businesses' == val.strip():
        return 1
    if 'All Non-Essential Businesses' == val.strip():
        return 2
    return 0

def C3(val):
    val=str(val)
    if  'Other' == val.strip():
        return 1
    elif '>10 People Prohibited' == val.strip():
        return 3
    elif 'All Gatherings Prohibited' == val.strip():
        return 4
    return 0

def C4(val):
    val=str(val)
    if 'High-risk Groups' == val.strip() or 'Other' == val.strip():
        return 1
    if 'Statewide' == val.strip():
        return 2
    return 0

def C5(val):
    val=str(val)
    if 'From Certain States' == val.strip() or 'ALL Air Travelers' == val.strip() or 'Other' == val.strip():
        return 1
    if 'All Travelers' == val.strip():
        return 2
    return 0

def C6(val):
    val=str(val)
    if  'Other' == val.strip():
        return 1
    elif 'Limited On-Site Service' == val.strip():
        return 2
    elif 'Closed Except for Takeout/Delivery' == val.strip():
        return 3
    return 0

def C7(val):
    val=str(val)
    if 'Yes' == val.strip() :
        return 1
    return 0

def C8(val):
    val=str(val)
    if 'Restrict' == val.strip():
        return 1
    if 'Yes' == val.strip():
        return 2
    return 0

def H1(val):
    val=str(val)
    if 'State-Insurer Agreement' == val.strip():
        return 1
    if 'State Requires' == val.strip():
        return 2
    return 0

def H2(val):
    val=str(val)
    if 'State-Insurer Agreement' == val.strip():
        return 1
    if 'State Requires' == val.strip():
        return 2
    return 0

def H3(val):
    val=str(val)
    if 'For COVID-19 Testing' == val.strip():
        return 1
    if 'For COVID-19 Testing and Treatment' == val.strip():
        return 2
    return 0

def H4(val):
    val=str(val)
    if 'State Requires' == val.strip():
        return 1
    return 0

def H5(val):
    val=str(val)
    if 'COVID-19 Diagnosis/Impacts Only' == val.strip():
        return 1
    if 'All Policies' == val.strip():
        return 2
    return 0

def H6(val):
    val=str(val)
    if 'Yes' == val.strip() :
        return 1
    return 0

def H7(val):
    val=str(val)
    if 'Approved' == val.strip() :
        return 1
    return 0

def H8(val):
    val=str(val)
    if 'Proposed' == val.strip():
        return 1
    if 'Enacted' == val.strip():
        return 2
    return 0

def O1(val):
    val=str(val)
    if val.strip()!='nan':
        return -1
    return 0

stringency_index=[]
for index,row in state_index.iterrows():  
    stringency_index.append(C1(row[1])+C2(row[2])+C3(row[3])+C4(row[4])+C5(row[5])+C6(row[6])+C7(row[7])+C8(row[8])+H1(row[9])+H2(row[10])+H3(row[11])+H4(row[12])+H5(row[13])+H6(row[14])+H7(row[15])+H8(row[16])+O1(row[17]))

state_index['stringency_index']=stringency_index
state_index.head(3)
    

,state_name,school_closures,non-essential_business_closures,large_gatherings_ban,stay_at_home_order,mandatory_quarantine_for_travelers,bar/restaurant_limits,primary_election_postponement,daycares,waive_cost_sharing_for_covid-19_treatment,free_cost_vaccine_when_available,state_requires_waiver_of_prior_authorization_requirements,early_prescription_refills,premium_payment_grace_period,marketplace_special_enrollment_period(SEP),section_1135_waiver,paid_sick_leave,state_is_easing_social_distancing_measures,stringency_index
0,Alabama,Closed for School Year,All Non-Essential Businesses,>10 People Prohibited,Statewide,NaN,Closed Except for Takeout/Delivery,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Approved,NaN,NaN,16
1,Alaska,Closed for School Year,All Non-Essential Businesses,All Gatherings Prohibited,Statewide,All Travelers,Closed Except for Takeout/Delivery,NaN,Yes,NaN,NaN,NaN,State Requires,All Policies,NaN,Approved,NaN,NaN,21
2,Arizona,Closed for School Year,All Non-Essential Businesses,>10 People Prohibited,Statewide,From Certain States,Closed Except for Takeout/Delivery,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Approved,Enacted,NaN,18


Calculate state government response stringency index on 2020/05/04.

In [41]:
state_index2_df=pd.read_csv('./processed_data/state_responses_05-04.csv')
state_index2=state_index2_df[['state_name','school_closures','non-essential_business_closures','large_gatherings_ban','stay_at_home_order',
                            'mandatory_quarantine_for_travelers','restaurant_limits','primary_election_postponement',
                           'daycares','waive_cost_sharing_for_covid-19_treatment','free_cost_vaccine_when_available','state_requires_waiver_of_prior_authorization_requirements',
                           'early_prescription_refills','premium_payment_grace_period','marketplace_special_enrollment_period(SEP)','section_1135_waiver','paid_sick_leave'
                            ,'state_is_easing_social_distancing_measures']]

state_index2.head(3)

,state_name,school_closures,non-essential_business_closures,large_gatherings_ban,stay_at_home_order,mandatory_quarantine_for_travelers,restaurant_limits,primary_election_postponement,daycares,waive_cost_sharing_for_covid-19_treatment,free_cost_vaccine_when_available,state_requires_waiver_of_prior_authorization_requirements,early_prescription_refills,premium_payment_grace_period,marketplace_special_enrollment_period(SEP),section_1135_waiver,paid_sick_leave,state_is_easing_social_distancing_measures
0,Alabama,Closed for School Year,Some Non-Essential Businesses Permitted to Reo...,>10 People Prohibited,Lifted,NaN,Closed Except for Takeout/Delivery,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Approved,NaN,Yes
1,Alaska,Closed for School Year,Some Non-Essential Businesses Permitted to Reo...,Expanded to 20+ People Prohibited,Lifted,All Travelers,Reopened to Dine-in Service with Capacity Limits,NaN,Yes,NaN,NaN,NaN,State Requires,All Policies,NaN,Approved,NaN,Yes
2,Arizona,Closed for School Year,All Non-Essential Businesses Closed,>10 People Prohibited,Statewide,From Certain States,Closed Except for Takeout/Delivery,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Approved,Enacted,NaN


In [43]:
def C1_2(val):
    val=str(val)
    if 'Recommended' in val:
        return 1
    if 'Closed' in val:
        return 2
    return 0

def C2_2(val):
    val=str(val)
    if 'Some Non-Essential Businesses Closed' == val.strip() or 'All Non-Essential Businesses Permitted to Reopen with Reduced Capacity'==val.strip() or 'Some Non-Essential Businesses Closed'==val.strip():
        return 1
    elif 'Some Non-Essential Businesses Permitted to Reopen with Reduced Capacity'==val.strip():
        return 2
    elif 'All Non-Essential Businesses Closed' == val.strip():
        return 3
    return 0

def C3_2(val):
    val=str(val)
    if  'Other' == val.strip() or 'Expanded to >25 People Prohibited'==val.strip() or 'Expanded to 25+ People Prohibited'==val.strip():
        return 1
    elif 'Expanded to 20+ People Prohibited' == val.strip():
        return 2
    elif 'Expanded to >10 People Prohibited' == val.strip() or '>10 People Prohibited'==val.strip():
        return 3
    elif 'All Gatherings Prohibited' == val.strip():
        return 4
    return 0

def C4_2(val):
    val=str(val)
    if 'High-risk Groups' == val.strip() or 'Other' == val.strip() or 'Rolled Back to High Risk Groups'==val.strip():
        return 1
    if 'Statewide' == val.strip():
        return 2
    return 0

def C5_2(val):
    val=str(val)
    if 'From Certain States' == val.strip() or 'ALL Air Travelers' == val.strip() or 'Other' == val.strip() or 'Rolled Back to Certain States'==val.strip():
        return 1
    if 'All Travelers' == val.strip():
        return 2
    return 0

def C6_2(val):
    val=str(val)
    if 'Reopened to Dine-in Service' == val.strip():
        return 1
    elif 'Reopened to Dine-in Service with Capacity Limits'==val.strip() or 'Limited Dine-in Service'==val.strip():
        return 2
    elif 'Closed Except for Takeout/Delivery' == val.strip():
        return 3
    return 0

def C7_2(val):
    val=str(val)
    if 'Yes' == val.strip() :
        return 1
    elif 'Canceled'==val.strip():
        return 2
    return 0

def C8_2(val):
    val=str(val)
    if 'Restrict' == val.strip():
        return 1
    if 'Yes' == val.strip():
        return 2
    return 0

def H1_2(val):
    val=str(val)
    if 'State-Insurer Agreement' == val.strip():
        return 1
    if 'State Requires' == val.strip():
        return 2
    return 0

def H2_2(val):
    val=str(val)
    if 'State-Insurer Agreement' == val.strip():
        return 1
    if 'State Requires' == val.strip():
        return 2
    return 0

def H3_2(val):
    val=str(val)
    if 'For COVID-19 Testing' == val.strip():
        return 1
    if 'For COVID-19 Testing and Treatment' == val.strip():
        return 2
    return 0

def H4_2(val):
    val=str(val)
    if 'State Requires' == val.strip():
        return 1
    return 0

def H5_2(val):
    val=str(val)
    if 'COVID-19 Diagnosis/Impacts Only' == val.strip():
        return 1
    if 'All Policies' == val.strip():
        return 2
    return 0

def H6_2(val):
    val=str(val)
    if 'Yes' == val.strip() :
        return 1
    return 0

def H7_2(val):
    val=str(val)
    if 'Approved' == val.strip() :
        return 1
    return 0

def H8_2(val):
    val=str(val)
    if 'Proposed' == val.strip():
        return 1
    if 'Enacted' == val.strip():
        return 2
    return 0

def O1_2(val):
    val=str(val)
    if val.strip()!='nan':
        return -1
    return 0

stringency_index2=[]
for index,row in state_index2.iterrows():  
    stringency_index2.append(C1_2(row[1])+C2_2(row[2])+C3_2(row[3])+C4_2(row[4])+C5_2(row[5])+C6_2(row[6])+C7_2(row[7])+C8_2(row[8])+H1_2(row[9])+H2_2(row[10])+H3_2(row[11])+H4_2(row[12])+H5_2(row[13])+H6_2(row[14])+H7_2(row[15])+H8_2(row[16])+O1_2(row[17]))

state_index2['stringency_index']=stringency_index2
state_index2.head(3)
    

,state_name,school_closures,non-essential_business_closures,large_gatherings_ban,stay_at_home_order,mandatory_quarantine_for_travelers,restaurant_limits,primary_election_postponement,daycares,waive_cost_sharing_for_covid-19_treatment,free_cost_vaccine_when_available,state_requires_waiver_of_prior_authorization_requirements,early_prescription_refills,premium_payment_grace_period,marketplace_special_enrollment_period(SEP),section_1135_waiver,paid_sick_leave,state_is_easing_social_distancing_measures,stringency_index
0,Alabama,Closed for School Year,Some Non-Essential Businesses Permitted to Reo...,>10 People Prohibited,Lifted,NaN,Closed Except for Takeout/Delivery,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Approved,NaN,Yes,12
1,Alaska,Closed for School Year,Some Non-Essential Businesses Permitted to Reo...,Expanded to 20+ People Prohibited,Lifted,All Travelers,Reopened to Dine-in Service with Capacity Limits,NaN,Yes,NaN,NaN,NaN,State Requires,All Policies,NaN,Approved,NaN,Yes,15
2,Arizona,Closed for School Year,All Non-Essential Businesses Closed,>10 People Prohibited,Statewide,From Certain States,Closed Except for Takeout/Delivery,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Approved,Enacted,NaN,19
